In [1]:
import selenium
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import numpy as np
import time
from selenium.webdriver.common.action_chains import ActionChains
import re
import datetime
import json

In [2]:
driver = webdriver.Chrome()
url = "https://www.cvedetails.com/vulnerabilities-by-types.php"
driver.get(url)
time.sleep(2)

In [3]:
by_types = dict()
by_impacts = dict()

In [4]:
exploit_links = []
#########################
exploit_cves = []
exploit_categories = []
exploit_descriptions = []
exploit_scores = []
exploit_published_date = []
exploit_updated_date = []
exploit_EPSS = []

In [5]:
tables = driver.find_elements(By.CLASS_NAME, "table-responsive")

types_table = tables[0]
types_thead = types_table.find_element(By.TAG_NAME, "thead")
types_headers = types_thead.find_elements(By.TAG_NAME, "th")[1:]
types_headers = [header.text for header in types_headers]
types_tbody = types_table.find_element(By.TAG_NAME, "tbody")
types_links_row = types_tbody.find_elements(By.TAG_NAME, "tr")[-1]
types_links = types_links_row.find_elements(By.TAG_NAME, "a")
types_links = [link.get_attribute("href") for link in types_links]

for header, link in zip(types_headers, types_links):
   by_types[header] = link

impacts_table = tables[1]
impacts_thead = impacts_table.find_element(By.TAG_NAME, "thead")
impacts_headers = impacts_thead.find_elements(By.TAG_NAME, "th")[1:]
impacts_headers = [header.text for header in impacts_headers]
impacts_tbody = impacts_table.find_element(By.TAG_NAME, "tbody")
impacts_links_row = impacts_tbody.find_elements(By.TAG_NAME, "tr")[-1]
impacts_links = impacts_links_row.find_elements(By.TAG_NAME, "a")
impacts_links = [link.get_attribute("href") for link in impacts_links]

for header, link in zip(impacts_headers, impacts_links):
   by_impacts[header] = link

print(by_types)
print(by_impacts)

In [6]:
#table 1
for t_type, t_link in by_types.items():
   driver.get(t_link)
   paginagion = driver.find_element(By.ID, "pagingt")
   pages_num = paginagion.find_elements(By.TAG_NAME, "a")[-1].text
   link_props = driver.find_element(By.CSS_SELECTOR, 'link[rel="canonical"]').get_attribute("href")
   sha_pattern = r"sha=([\d\w]+)"
   trc_pattern = r"trc=(\d+)"
   op_pattern = r"page=\d+&(\w+)"
   sha = re.findall(sha_pattern, link_props)[0]
   trc = re.findall(trc_pattern, link_props)[0]
   op = re.findall(op_pattern, link_props)[0]

   for page_num in range(1,int(pages_num) + 1):
       url = f'{t_link}?page={str(page_num)}&{op}=1&order=2&trc={str(trc)}&sha={sha}'
       driver.get(url)
       time.sleep(0.2)
       search_results = driver.find_element(By.ID, "searchresults")
       
       links_elements = search_results.find_elements(By.TAG_NAME, "a")
       results_num = len(links_elements)
       links = [link.get_attribute("href") for link in links_elements]
       exploit_links.extend(links)
       

       
#table 2        
for i_type, i_link in by_impacts.items():
   driver.get(i_link)
   paginagion = driver.find_element(By.ID, "pagingt")
   pages_num = paginagion.find_elements(By.TAG_NAME, "a")[-1].text
   link_props = driver.find_element(By.CSS_SELECTOR, 'link[rel="canonical"]').get_attribute("href")
   sha_pattern = r"sha=([\d\w]+)"
   trc_pattern = r"trc=(\d+)"
   op_pattern = r"page=\d+&(\w+)"
   sha = re.findall(sha_pattern, link_props)[0]
   trc = re.findall(trc_pattern, link_props)[0]
   op = re.findall(op_pattern, link_props)[0]

   for page_num in range(1,int(pages_num) + 1):
       url = f'{i_link}?page={str(page_num)}&{op}=1&order=2&trc={str(trc)}&sha={sha}'
       driver.get(url)
       time.sleep(0.2)
       search_results = driver.find_element(By.ID, "searchresults")
       
       links_elements = search_results.find_elements(By.TAG_NAME, "a")
       results_num = len(links_elements)
       links = [link.get_attribute("href") for link in links_elements]
       exploit_links.extend(links)
       
       
with open('exploit_links.txt', 'w') as file:
   for link in exploit_links:
       file.write(link + '\n')
       
       cves = [link.text for link in links_elements]
       exploit_cves.extend(cves)
       
       types = [t_type]*results_num
       exploit_types.extend(types)
       
       impacts = [None]*results_num
       exploit_impacts.extend(impacts)
       
       descriptions = search_results.find_elements(By.CLASS_NAME, "cvesummarylong")
       descriptions = [des.text for des in descriptions]
       exploit_descriptions.extend(descriptions)
       
       max_base_scores = search_results.find_elements(By.TAG_NAME, "td")[1::8]
       max_base_scores = [score.text for score in max_base_scores]
       exploit_max_base_scores.extend(max_base_scores)
       
       published_date = search_results.find_elements(By.TAG_NAME, "td")[3::8]
       published_date = [date.text for date in published_date]
       exploit_published_date.extend(published_date)

       updated_date = search_results.find_elements(By.TAG_NAME, "td")[5::8]
       updated_date = [date.text for date in updated_date]
       exploit_updated_date.extend(exploit_updated_date)
       
       EPSS = search_results.find_elements(By.TAG_NAME, "td")[7::8]
       EPSS = [e.text for e in EPSS]
       exploit_EPSS.extend(EPSS)


In [7]:
# opening the file in read mode
my_file = open("exploit_links.txt", "r")
 
# reading the file
data = my_file.read()
 
# replacing end splitting the text 
# when newline ('\n') is seen.
exploit_links = data.split("\n")
my_file.close()

In [8]:
links_no_duplicates = []
[links_no_duplicates.append(x) for x in exploit_links if x not in links_no_duplicates]

In [9]:
with open('exploit_links_no_dups.txt', 'w') as file:
   for link in links_no_duplicates:
       file.write(link + '\n')

In [10]:
# opening the file in read mode
my_file = open("exploit_links_no_dups.txt", "r")
  
# reading the file
data = my_file.read()
  
# replacing end splitting the text 
# when newline ('\n') is seen.
exploit_links_no_dups = data.split("\n")
my_file.close()

In [ ]:
current_date = datetime.date.today()
formatted_date = current_date.strftime("%d/%m/%Y")
i = 1
length = len(exploit_links_no_dups)

cve_pattern = r"CVE-\d+-\d+"

date_pattern = r"\d+-\d+-\d+.*\d+:\d+:\d+"

for link in exploit_links_no_dups:
    driver.get(link)
    time.sleep(1)
    
    print(f'{i} of {length}')
    
    try:
        cve = re.findall(cve_pattern, link)[0]
        exploit_cves.append(cve)
    except:
        exploit_cves.append(np.nan)
    
    
    try:
        categories = driver.find_elements(By.CLASS_NAME, 'ssc-vuln-cat')
        categories = [category.text for category in categories]
        exploit_categories.append(categories)
    except:
        exploit_categories.append(np.nan)
        
        
    try:
        description = driver.find_element(By.CLASS_NAME, 'cvedetailssummary-text').text
        exploit_descriptions.append(description)
    except:
        exploit_descriptions.append(np.nan)
    
    
    try:
        dates = driver.find_elements(By.CLASS_NAME,'d-inline-block')
        published = dates[0]
        published_date = re.findall(date_pattern, published.text)[0]
        exploit_published_date.append(published_date)
    except:
        exploit_published_date.append(np.nan)
        
        
    try:
        updated = dates[1]
        updated_date = re.findall(date_pattern, updated.text)[0]
        exploit_updated_date.append(updated_date)
    except:
        exploit_updated_date.append(np.nan)
        
        
    try:
        epss_scores = driver.find_elements(By.CLASS_NAME,'epssbox')
        epss1 = epss_scores[0].text
    except:
        epss1 = np.nan
        
    try:
        epss2 = epss_scores[1].text
    except:
        epss2 = np.nan    
        
        
    EPSS_details = {"Date:" : formatted_date,
                    "Probability of exploitation activity in the next 30 days:" : epss1,
                    "Percentile, the proportion of vulnerabilities that are scored at or less:" : epss2}
    
    exploit_EPSS.append(EPSS_details)
    
    try:
        scores_details = {"Base Score": [],
                          "Base Severity": [],
                          "CVSS Vector": [],
                          "Exploitability Score": [],
                          "Impact Score": [],
                          "Source": []} 
        score_table = driver.find_element(By.CLASS_NAME, "table")
        tbody = score_table.find_element(By.TAG_NAME, "tbody")
        rows = tbody.find_elements(By.TAG_NAME, "tr")
        for row in rows:
            fields = row.find_elements(By.TAG_NAME, "td")
            for field, key, value in zip(fields, scores_details.keys(), scores_details.values()):
                if field.text != "":
                    scores_details[key].append(field.text)
        exploit_scores.append(scores_details)
    except:
        exploit_scores.append(np.nan)
        
    i += 1

In [22]:
with open('exploit_cves.txt', 'w', encoding='utf-8') as file:
    for cve in exploit_cves:
        file.write(cve + '\n')

with open('exploit_published_date.txt', 'w', encoding='utf-8') as file:
    for date in exploit_published_date:
        file.write(str(date) + '\n')

with open('exploit_updated_date.txt', 'w', encoding='utf-8') as file:
    for date in exploit_updated_date:
        file.write(str(date) + '\n')

with open('exploit_categories.txt', 'w', encoding='utf-8') as file:
    for cat in exploit_categories:
        file.write(str(cat) + '\n')

with open('exploit_scores.txt', 'w', encoding='utf-8') as file:
    for score in exploit_scores:
        file.write(str(score) + '\n')

with open('exploit_EPSS.txt', 'w', encoding='utf-8') as file:
    for epss in exploit_EPSS:
        file.write(str(epss) + '\n')

with open('exploit_descriptions.txt', 'w', encoding='utf-8') as file:
    for des in exploit_descriptions:
        file.write(str(des) + '\n')


In [13]:
# Create a dictionary for the data
data = {
    'CVE': exploit_cves,
    'PUBLISHED': exploit_published_date,
    'UPDATED': exploit_updated_date,
    'CATEGORY': exploit_categories,
    'BASE SCORE (TABLE)': exploit_scores,
    'EPSS': exploit_EPSS,
    'DESCRIPTION': exploit_descriptions
}

# Convert the dictionary to a DataFrame
df = pd.DataFrame(data)

# Saving the DataFrame to a CSV file
df.to_csv('exploits_data.csv', index=False)

In [31]:
# Specify the JSON file name
json_file = "exploits_data.json"

# Write the dictionary to the JSON file
with open(json_file, "w") as file:
    json.dump(data, file)

print(f"Data has been saved to {json_file}")

Data has been saved to exploits_data.json


In [32]:
df

,CVE,PUBLISHED,UPDATED,CATEGORY,BASE SCORE (TABLE),EPSS,DESCRIPTION
0,CVE-1999-0002,1998-10-12 04:00:00,2009-01-26 05:00:00,[Overflow],"{'Base Score': ['10.0'], 'Base Severity': ['HI...","{'Date:': '19/09/2023', 'Probability of exploi...",Buffer overflow in NFS mountd gives root acces...
1,CVE-1999-0003,1998-04-01 05:00:00,2018-10-30 16:26:22,[Overflow],"{'Base Score': ['10.0'], 'Base Severity': ['HI...","{'Date:': '19/09/2023', 'Probability of exploi...",Execute commands as root via buffer overflow i...
2,CVE-1999-0004,1997-12-16 05:00:00,2018-10-12 21:29:02,[Overflow],"{'Base Score': ['5.0'], 'Base Severity': ['MED...","{'Date:': '19/09/2023', 'Probability of exploi...","MIME buffer overflow in email clients, e.g. So..."
3,CVE-1999-0005,1998-07-20 04:00:00,2008-09-09 12:33:31,[Overflow],"{'Base Score': ['10.0'], 'Base Severity': ['HI...","{'Date:': '19/09/2023', 'Probability of exploi...",Arbitrary command execution via IMAP buffer ov...
4,CVE-1999-0006,1998-07-14 04:00:00,2008-09-09 12:33:31,[Overflow],"{'Base Score': ['10.0'], 'Base Severity': ['HI...","{'Date:': '19/09/2023', 'Probability of exploi...",Buffer overflow in POP servers based on BSD/Qu...
...,...,...,...,...,...,...,...
151830,CVE-2022-35980,2022-08-12 18:15:14,2022-08-16 15:32:31,[Information leak],"{'Base Score': ['7.5', '7.5'], 'Base Severity'...","{'Date:': '19/09/2023', 'Probability of exploi...",OpenSearch Security is a plugin for OpenSearch...
151831,CVE-2022-36800,2022-08-03 03:15:08,2022-08-09 17:15:20,[Information leak],"{'Base Score': ['4.3'], 'Base Severity': ['MED...","{'Date:': '19/09/2023', 'Probability of exploi...",Affected versions of Atlassian Jira Service Ma...
151832,CVE-2022-36834,2022-08-05 16:15:15,2022-09-20 20:47:59,[Information leak],"{'Base Score': ['3.3', '5.0'], 'Base Severity'...","{'Date:': '19/09/2023', 'Probability of exploi...",Exposure of Sensitive Information vulnerabilit...
151833,CVE-2022-36835,2022-08-05 16:15:15,2022-10-27 17:29:47,[Information leak],"{'Base Score': ['3.3', '3.3'], 'Base Severity'...","{'Date:': '19/09/2023', 'Probability of exploi...",Implicit Intent hijacking vulnerability in Sam...
